# Benchmark Data Set

In this notebook, we explain how we create the benchmark testset for comparing different architectures and approaches. We will create two data sets consisting of 10.000 extended Gaussian sources with varying sampling masks. One of them with additional noise corruption on the images, the other one without.

* Fourier
* Amp_Phase
* Interpolation

In [1]:
import toml
from radionets.simulations.utils import read_config, check_outpath, adjust_outpath, add_noise
from radionets.simulations.gaussians import create_grid, create_ext_gauss_bundle
from radionets.simulations.simulate import sample_fft_images
from radionets.dl_framework.data import save_fft_pair_list
import numpy as np
import h5py
from tqdm import tqdm
from pathlib import Path
import re

### Configuration

In [2]:
config = toml.load("./.rc/benchmark/sim_benchmark.toml")
sim_conf = read_config(config)

# specify data_path
sim_conf["data_path"] = "../../noise/benchmark_data"

sim_fft, sim_sampled = check_outpath(
    sim_conf["data_path"],
    data_format=config["paths"]["data_format"],
    quiet=config["mode"]["quiet"],
)

Create fft_images from gaussian data set! 

Adding extended gaussian sources.
Found existing fft_files!
Overwriting old fft_files!


### Start Simulation

In [3]:
data_path=sim_conf["data_path"]
option="test"
num_bundles=sim_conf["bundles_test"]
bundle_size=sim_conf["bundle_size"]
img_size=sim_conf["img_size"]
num_comp_ext=sim_conf["num_components"]
num_pointlike=sim_conf["num_pointlike_gaussians"]
num_pointsources=sim_conf["num_pointsources"]
noise=sim_conf["noise"]
source_list=sim_conf["source_list"]

In [4]:
np.random.seed(1337)

In [5]:
# create source images
for i in tqdm(range(num_bundles)):
    grid = create_grid(img_size, bundle_size)
    ext_gaussian = 0
    pointlike = 0
    pointsource = 0
    list_sources = 0

    if num_comp_ext is not None:
        ext_gaussian = create_ext_gauss_bundle(grid)
    if num_pointlike is not None:
        pointlike = create_gauss(
            grid[:, 0], bundle_size, num_pointlike, True, source_list
        )
    if num_pointsources is not None:
        pointsource = gauss_pointsources(grid[:, 0], bundle_size, num_pointsources)
    if source_list:
        list_sources = pointlike[1]
        pointlike = pointlike[0]

    bundle = ext_gaussian + pointlike + pointsource
    images = bundle.copy()

    path = adjust_outpath(data_path, "/source_" + option)

    with h5py.File(path, "w") as hf:
        hf.create_dataset("sources", data=bundle)

100%|██████████| 1/1 [00:00<00:00, 54.28it/s]


In [6]:
# use source images and create fft versions without noise

# get source paths
source_path = Path(sim_conf["data_path"])
s_paths = np.array([x for x in source_path.iterdir()])
s_paths = [
    path for path in s_paths if re.findall(f"source_test", path.name)
]

# create and save fft without noise
for p in s_paths:
    f = h5py.File(p, "r")
    s = np.array(f["sources"])
    images = s.copy()
    if noise:
        images = add_noise(images)
    bundle_fft = np.array([np.fft.fftshift(np.fft.fft2(img)) for img in images])
    out = p.parent/"clean"/p.name.replace("source", "fft")
    out.parent.mkdir(exist_ok=True)
    save_fft_pair_list(out, bundle_fft, bundle, 0)
    f.close()

# create and save fft with noise
for p in s_paths:
    f = h5py.File(p, "r")
    s = np.array(f["sources"])
    images = s.copy()
    images = add_noise(images)
    bundle_fft = np.array([np.fft.fftshift(np.fft.fft2(img)) for img in images])
    out = p.parent/"noise"/p.name.replace("source", "fft")
    out.parent.mkdir(exist_ok=True)
    save_fft_pair_list(out, bundle_fft, bundle, 0)
    f.close()

### Sampling Frequencies

In [8]:
sim_conf["data_path"] = "../../noise/benchmark_data/clean/"
sample_fft_images(sim_conf)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
  0%|          | 0/2 [00:00<?, ?it/s]


 Sampling train data set.


 No train data set fft images available.


 Sampling valid data set.


 No valid data set fft images available.


 Sampling test data set.



100%|██████████| 2/2 [00:01<00:00,  1.20it/s]


In [9]:
sim_conf["data_path"] = "../../noise/benchmark_data/noise/"
sample_fft_images(sim_conf)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
  0%|          | 0/2 [00:00<?, ?it/s]


 Sampling train data set.


 No train data set fft images available.


 Sampling valid data set.


 No valid data set fft images available.


 Sampling test data set.



100%|██████████| 2/2 [00:01<00:00,  1.28it/s]
